# Initialize

## from tracks

In [ ]:
%sx mount /media/data
# saving the track files to the local disk: reloading the analyses occurs faster
from trackanalysis import *

TRACKS = TracksDict("/media/data/sirius/Andreas/ssHP6_Sequencing_20180102/*/*.ana", match = ".*OR3_mix_(...)_.*")
TRACKS['ref'] = "/media/data/sirius/Andreas/ssHP6_Sequencing_20180102/Reference_oligo/Test_020_044_ssHP6_OR3-10_100nM_OR3-11_20nM_after_saturation.ana"

# we are now going to modify the paths as they were set on another system
# we'll need to load the data for the modified paths to be taken into account
patt = re.compile(".*_OR3_mix_(.*?)_.*")
def _modpath(x):
    # modify the list of paths to match the computer's paths
    # as it happens, only one path (.trk) is expected here
    if patt.match(x[0]):
        # oligos
        return [TRACKS[patt.match(x[0]).group(1)].path[0][:-3]+"trk"]
    # ref
    return [TRACKS['ref'].path[0][:-3]+"trk"]
TRACKS.load(path=_modpath) # load and modify the paths

# can now save
TRACKS = TRACKS.save("/home/pol/Documents/tracks/rnasequencing/ssHP6_Sequencing_20180102/")

## from pk

In [ ]:
# reading the track files
from trackanalysis import *

TRACKS = TracksDict("/home/pol/Documents/tracks/rnasequencing/ssHP6_Sequencing_20180102/*.pk")

# Add & remove specific tasks as wanted
for i in TRACKS.values():
    i.tasks.cleaning  = None
    i.tasks.selection = None
TRACKS['GCA'].tasks.subtraction            = 0, 1, 4, 6, 8, 9
TRACKS['GCA'].tasks.cleaning = Tasks.cleaning(maxsaturation = 100.)
for i in TRACKS.values():
    i.tasks.cleaning = Tasks.cleaning(minpopulation = 60.) # bead 18, track GGC has many missing cycles

# the shelf will save computations to the hard drive
# this makes reloading an analysis much quicker
SHELF = LazyShelf("/home/pol/Documents/tracks/rnasequencing/ssHP6_Sequencing_20180102/20180327_rnasequencing_201801_biasperoligo.shelf")

# Sequences: I'm expecting:
# * full: the full sequence
# * target: the part to be sequenced
# * oligo: the reference oligo
SEQ   = dict(sequences.read("/home/pol/Documents/tracks/rnasequencing/ssHP6_all.txt"))

# Viewing Data

In [ ]:
%%opts Curve(alpha=0.5)
TRACKS.cleancycles

# Alignments

In [ ]:
# alignment code
from aligningexperiments import *

def hppeaks(data, delta, seq = SEQ):
    # return a *gray* scatter plot with the theoretical positions
    # I'm expecting that the sequenc
    xvals = []
    yvals = []
    
    tgt   = [seq['full'].index(seq['target']), seq['full'].index(seq['target'])+len(seq['target'])]
    for i in data.track.unique():
        if i == 'ref':
            oli = seq['oligo']
        else:
            oli = sequences.Translator.reversecomplement(i)
            
        oli   = sequences.peaks(seq['full'], oli)
        if i != 'ref':
            oli   = oli[oli['position'] >= tgt[0]]
            oli   = oli[oli['position'] <= tgt[1]]
        oli   = oli['position'][oli['orientation']]
        xvals.append([i]*len(oli))
        yvals.append(oli)
    return hv.Scatter((np.concatenate(xvals), np.concatenate(yvals)+delta),
                      label = 'oligo')(style =  dict(color = 'gray', alpha = .5, size = 5))

def displayalignedbeads(data,
                        ref       = 18,
                        normalize = True,
                        discarded = None,
                        masks     = None,
                        stretch   = Range(1., .15, .03),
                        bias      = Range(0., 0.1, 0.01),
                        **kwa):
    # align all good beads in a track file
    align = PeaksAlignment(**kwa)
    if normalize is False:
        align.refalign = None
    else:
        align.refalign.stretch = stretch
        align.refalign.bias    = bias

    data  = align(data, ref, discarded = discarded, masks = masks)
    out   = align.display(data, align = False)
    if align.hpalign is None:
        return out
    
    tgt   = [SEQ['full'].index(SEQ['target']), SEQ['full'].index(SEQ['target'])+len(SEQ['target'])]
    oli   = sequences.Translator.reversecomplement(data.track.unique()[0])
    oli   = sequences.peaks(SEQ['full'], oli)
    oli   = oli[oli['position'] >= tgt[0]]
    oli   = oli[oli['position'] <= tgt[1]]
    oli   = oli['position'][oli['orientation']]

    ref   = sorted(list(sequences.peaks(SEQ['full'], SEQ['oligo'])['position'])+ tgt)
    style = dict(color = 'gray', alpha = .5, size = 5)
    out   = align.hpindisplay(data, oli, label = 'oligo', group = 'ref',
                              style = dict(marker='square', **style))*out
    out   = align.hpindisplay(data, ref, label = 'reference', group = 'ref',
                              style = dict(marker='diamond', **style))*out
    return out

# create the data
SHELF['DATA2'] = lambda: createpeaks(TRACKS)
DATA          = SHELF['DATA2']

In [ ]:
# display the beads per number of tracks
# good beads have many track and low resolution values
hv.BoxWhisker(DATA.assign(resolution = DATA.resolution*1e3), ["trackcount", "bead"], "resolution")

In [ ]:
%%opts Scatter[show_legend=False]
# Check the alignments
align = PeaksAlignment()
align.refalign.stretch = Range(1., .05, .02)
align.refalign.bias    = Range(0., .01, .002)
align.refalign.pivot   = Pivot.absolute
MASKS = {}
DELTA = {}
def _run(bead):
    align.hpalign.peaks = align.hpalign.topeaks(SEQ['full'], SEQ['oligo'])
    idelta = DELTA.get(bead, 0)
    delta  = align.hpalign.peaks[idelta]
    align.hpalign.peaks =  align.hpalign.peaks[idelta:]-delta
    return (align.display(DATA[DATA.bead == bead], 'ref', masks= MASKS.get(bead, None))
            *hppeaks(DATA, -delta))

_run(18)

In [ ]:
# save the csv
PEAKS = (lambda i: align(DATA[DATA.bead == i], 'ref', masks = MASKS.get(i, None))
         )(18)
BARR = PEAKS[lambda x: x.track=='ref'].peakposition.unique()[[3,4]] + [3e-3, -3e-3]
GOOD = (PEAKS
        [lambda x: x.peakposition > BARR[0]]
        [lambda x: x.peakposition < BARR[1]])
pd.DataFrame({sequences.Translator.reversecomplement(i).lower(): pd.Series(GOOD[lambda x: x.track == i].peakposition.unique())
              for i in set(GOOD.track.unique()) - {'ref'}}).to_csv("/tmp/data.csv")